In [1]:
import xarray as xr
import os
import pystac_client
from dask.distributed import Client
import dask
import json
import pandas as pd
import stackstac

In [2]:
bbox = {
        "type": "Polygon",
        "coordinates": [
            [[-121.85308124466923,46.94373134954458],
            [-121.85308124466923,46.785391494446145],
            [-121.63845508457872,46.785391494446145],
            [-121.63845508457872,46.94373134954458],
            [-121.85308124466923,46.94373134954458]]]
    }

In [3]:
# We use the api from element84 to query the data
URL = "https://earth-search.aws.element84.com/v1"
catalog = pystac_client.Client.open(URL)

In [4]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=bbox,
    query={"eo:cloud_cover": {"lt": 10}}, # less than 10% cloud cover
)

# Check how many items were returned
items = search.item_collection()
print(f"Returned {len(items)} Items")

Returned 1455 Items


In [13]:
# create xarray dataset without loading data
sentinel2_stack = stackstac.stack(items, assets=['nir'], epsg=4326)

In [14]:
sentinel2_stack_snowoff = sentinel2_stack.where((sentinel2_stack.time.dt.month >= 6) & (sentinel2_stack.time.dt.month <= 11))

In [15]:
sentinel2_stack_snowoff

<xarray.DataArray 'stackstac-d8c585e1b20aea6c7569e1fc23498b87' (time: 1455,
                                                                band: 1,
                                                                y: 328663,
                                                                x: 1078224)> Size: 4PB
dask.array<where, shape=(1455, 1, 328663, 1078224), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/53)
  * time                                     (time) datetime64[ns] 12kB 2017-...
    id                                       (time) <U24 140kB 'S2A_10TET_201...
  * band                                     (band) <U3 12B 'nir'
  * x                                        (x) float64 9MB -123.0 ... -120.2
  * y                                        (y) float64 3MB 47.85 ... 45.93
    view:sun_azimuth                         (time) float64 12kB 164.8 ... 153.3
    ...                                       ...
    proj:centroid                            (time) object 12kB None ... {'la...
    s2:tile_id                               (time) object 12kB None ... 'S2A...
    view:azimuth                             (time) object 12kB None ... 286....
    proj:shape                               object 8B {10980}
    gsd                                      int64 8B 10
    epsg                                     int64 8B 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-123.00025421582134, 45.933...
    crs:            epsg:4326
    transform:      | 0.00, 0.00,-123.00|\n| 0.00,-0.00, 47.85|\n| 0.00, 0.00...
    resolution_xy:  (2.599726775955742e-06, 5.842440801457654e-06)

In [ ]:
# filter to specified month range
sentinel2_stack_snowoff = sentinel2_stack.where((sentinel2_stack.time.dt.month >= 6) & (sentinel2_stack.time.dt.month <= 11), drop=True)

In [ ]:
# select first image of each month
period_index = pd.PeriodIndex(sentinel2_stack_snowoff['time'].values, freq='M')
sentinel2_stack_snowoff.coords['year_month'] = ('time', period_index)
first_image_indices = sentinel2_stack_snowoff.groupby('year_month').apply(lambda x: x.isel(time=0))

In [ ]:
product_names = first_image_indices['s2:product_uri'].values.tolist()

In [ ]:
product_names

In [19]:
print(image_dates)

['2018-06-02', '2018-06-07', '2018-07-15', '2018-08-04', '2018-09-18', '2019-07-05', '2019-09-28', '2020-06-29', '2020-07-24', '2020-07-29', '2020-08-23', '2020-08-25', '2020-08-25', '2020-08-25', '2020-08-25', '2020-09-22', '2021-06-06', '2021-06-09', '2021-09-02', '2021-09-04', '2021-09-07', '2021-09-07', '2021-09-09', '2021-09-09', '2021-09-19', '2021-09-27', '2021-09-29', '2022-06-24', '2022-07-01', '2022-07-04', '2022-07-19', '2022-07-21', '2022-09-19', '2022-09-27', '2022-09-29', '2023-06-19', '2023-06-21', '2023-06-24', '2023-07-14', '2023-08-13', '2023-08-30', '2023-09-12']


In [20]:
image_dates = sentinel2_stack_snowoff.time.dt.strftime('%Y-%m-%d').values.tolist()

In [21]:
npairs = 3

In [22]:
pairs = []
for r in range(len(image_dates) - npairs):
    for s in range(1, npairs + 1 ):
      img1_date = image_dates[r]
      img2_date = image_dates[r+s]
      shortname = f'{img1_date}_{img2_date}'
      pairs.append({'img1_date': img1_date, 'img2_date': img2_date, 'name':shortname})
matrixJSON = f'{{"include":{json.dumps(pairs)}}}'
print(f'Number of Interferograms: {len(pairs)}')

with open(os.environ['GITHUB_OUTPUT'], 'a') as f:
    print(f'BURST_IDS={image_dates}', file=f)
    print(f'MATRIX_PARAMS_COMBINATIONS={matrixJSON}', file=f)

Number of Interferograms: 117


KeyError: 'GITHUB_OUTPUT'